In [1]:
import pandas as pd
import cassandra

In [2]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

# keyspace
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS sparkify 
                    WITH REPLICATION = 
                    { 'class' : 'SimpleStrategy', 
                      'replication_factor' : 1 }
                    """)

except Exception as e:
    print(e)
    
# connect to keyspace
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


In [3]:
query = "SELECT artist, song_title, song_duration FROM music_app_session_history WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [3]:
query = "SELECT artist, song_title, song_duration FROM music_app_session_history WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Reasons for creating the table: music_app_user_history
##### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- The query asks for particular columns (artist, song, user's first name, user's last name) given a particular user with *user_id* and in a particular session, with *session_id*, ordered by order in session with *item_in_session*
- So, we had to design your table for this query and also for fast reads
- As, two particular attributes are in 'WHERE' clause and we need to sort by another column, they (the combination of these three columns in order) were the first choices for 'PRIMARY KEY' as the combination of the two columns *user_id* and *session_id* will be used for partition and ordering based on *item_in_session*, so this will be the clustering column. Lastly combination of these three columns will create uniqueness needed for 'PRIMARY KEY' 
- The database construct
```
CREATE TABLE IF NOT EXISTS music_app_user_history
 (user_id int, session_id int, item_in_session int, artist text, 
 song text, user_first_name text, user_last_name text, 
 PRIMARY KEY ((user_id, session_id), item_in_session));
```

In [4]:
query = "SELECT artist, song, user_first_name, user_last_name FROM music_app_user_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user_first_name, row.user_last_name)


Keep On Keepin' On Down To The Bone Sylvie Cruz
Greece 2000 Three Drives Sylvie Cruz
Kilometer Sebastien Tellier Sylvie Cruz
Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Lonnie Gordon Sylvie Cruz


#### Reasons for creating the table: music_app_song_history
##### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- The query asks for particular columns (users) with user's first name and user's last name given a particular song with *song*
- So, we had to design your table for this query and also for fast reads
- As, one particular column is in 'WHERE' clause, we need to have this column in the 'PRIMARY KEY' but then it will not be unique, so we need uniqueness in terms of users also, then the first choice is *user_id* because ultimately we need user's first name and last name in *user_first_name*, *user_last_name*
- The database construct
```
CREATE TABLE IF NOT EXISTS music_app_song_history (song text, 
 user_id int, user_first_name text, user_last_name text, 
 PRIMARY KEY (song, user_id));
```

In [5]:
query = "SELECT user_first_name, user_last_name from music_app_song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name + " " + row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson
